In [1]:
# -*- coding: utf-8 -*-
import random
import re
import sys
import math
import numpy as np
import uncertainties.unumpy as unp
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from iminuit import Minuit
import yaml
from resample.bootstrap import bootstrap

In [2]:
run_n = 100#snakemake.params.run_n # Anzahl der Dateien
ordner = 'CCB'
Teilchenzahl = 10000000#snakemake.params.Teilchenzahl

In [3]:
def get_lines(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close
    return lines

myfunction = 'np.exp(a*z**2+ b*z + c)'
parameter_names=['a', 'b', 'c']

def epolynom(z,par):
    return np.exp(par[0]*z**2+ par[1]*z + par[2])

def unpepolynom(z,par):
    return unp.exp(par[0]*z**2+ par[1]*z + par[2])

def least_squares(par):
    mu = epolynom(xdata, par)
    yvar = yerr**2
    return sum((ydata - mu) ** 2 / yvar)

In [4]:
#### get all info from tdk files
for run in range(0,run_n):
        inputdatei = ordner + '-build/TDK_' + str(run) + '.txt'
        lines_name = 'lines_' + str(run)
        globals()[lines_name] = get_lines(inputdatei)

### get an array with the size we need
dose_array = np.zeros((run_n, len(lines_0)-3))

### read the lines from tdk files, and fill in the dose array
for run in range(0,run_n):
    for i in range(3, len(lines_0)):
        lines_name = 'lines_' + str(run)
        line = globals()[lines_name][i].replace(',', ' ')
        line = line.split()
        dose_array[run, i-3] = line[3]

In [5]:
dose_array = dose_array / Teilchenzahl
volumes = np.shape(dose_array)[1]
#actual_mean = np.zeros(volumes)
#actual_error = np.zeros(volumes)
bootstrap_mean = np.zeros(volumes)
bootstrap_error = np.zeros(volumes)
for volume in range(volumes):
    dose_in_volume = dose_array[:,volume]
    #actual_mean[volume] = np.mean(dose_in_volume)
    #actual_error[volume] = np.std(dose_in_volume)/np.sqrt(len(dose_in_volume))
    replicas = bootstrap(dose_in_volume)
    bootstrap_mean[volume] = np.mean(replicas)
    bootstrap_error[volume] = np.std(np.mean(replicas, axis=1))

In [6]:
## calculate mean, standard error and relative error
#dose_array_mean = np.mean(dose_array, axis=0)
#dose_array_ste = np.std(dose_array, axis = 0)/np.sqrt(len(dose_array))

dose_array_mean = bootstrap_mean
dose_array_ste = bootstrap_error

dose_array_rel = np.zeros(len(dose_array_mean))
for i in range(len(dose_array_rel)):
    if dose_array_mean[i] > 0:
        dose_array_rel[i] = dose_array_ste[i]/dose_array_mean[i] * 100

In [7]:
### get the z bins
z=np.zeros(len(lines_0)-3)
for i in range(3, len(lines_0)):
    line_0 = lines_0[i].replace(',', ' ')
    line_0 = line_0.split()
    z[i-3] = line_0[2]
#calculate the distance out of it
z = z+0.5

In [8]:
ydata = dose_array_mean[dose_array_mean > 0]
yerr = dose_array_ste[dose_array_mean > 0]
xdata = z[dose_array_mean > 0]
    
try:
    m = Minuit.from_array_func(least_squares,(-0.02, -0.2, -8.4), error=(-0.01, -0.1, -4.2), errordef=1)
    m.migrad()
    s_sq = m.fval / (len(ydata) - len(m.np_values()))
    fit_success = True
except:
    fit_success = False
    print('kein Fit')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in square
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in true_divide


In [9]:
#get some plotting
if fit_success == True:
    z_plot = np.linspace(0,10,100)
    D_plot = unpepolynom(z_plot, m.np_values())
    plt.errorbar(z_plot, unp.nominal_values(D_plot), xerr=None, yerr=None, fmt='r-', label = 'fit')


plt.errorbar(z[dose_array_mean > 0], dose_array_mean[dose_array_mean > 0], xerr= None, yerr=dose_array_ste[dose_array_mean > 0], fmt='k,', capsize =1, label=('TDK'))
plt.xlabel(r'Abstand in z Richtung / $\si{\milli\meter}$')
plt.ylabel('Dosis pro Zerfall / $\si{\gray}$')
plt.yscale('log')
plt.legend()
plt.savefig('Ergebnisse/TDK.pdf', bbox_inches="tight")
#plt.show()
plt.clf()

<Figure size 432x288 with 0 Axes>

In [10]:
###  print the fit parameter to a file
outputfile = 'Ergebnisse/TDK-Fitparameter.txt'
if fit_success==True:
    with open(outputfile, 'wt', encoding='utf-8') as out_file:
        out_file.write(myfunction + '\n')
        for i in range(len(m.np_values())):
            out_file.write( parameter_names[i] + '\t' + str(m.np_values()[i]) +'\t' + str(m.np_errors()[i]) + '\n')
        out_file.write('reduziertes Chi-Quadrat: ' + str(round(s_sq, 2)))
elif fit_success==False:
    with open(outputfile, 'wt', encoding='utf-8') as out_file:
        out_file.write('kein Fit')


outputfile = 'Ergebnisse/TDK-Werte.txt'
with open(outputfile, 'wt', encoding='utf-8') as out_file:
    out_file.write('z/mm \t Dosis/Gy \t Dosis error/Gy \t Dosis_error/%\n')
    for i in range(len(dose_array_mean)):
        out_file.write('{}\t'.format(z[i]))
        out_file.write('{:.2e}\t'.format(dose_array_mean[i]))
        out_file.write('{:.2e}\t'.format(dose_array_ste[i]))
        out_file.write('{:.2e}\n'.format(dose_array_rel[i]))